In [ ]:
################################################################################
# Author 1:      Firstname Lastname
# MatNr 1:       01234567
# Author 2:      Firstname Lastname
# MatNr 2:       01234567
# Author 3:      Jakob Marktl
# MatNr 3:       12335939
# File:          assignment2.ipynb
# Description: ... short description of the file ...
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pycountry
import pycountry_convert

Alles in einem File, programmieren

Robert:

David:

In [1]:
print("David hat was gemacht")

David hat was gemacht


Jakob:

In [ ]:
# Functions:
# Function for Bar Chart
def create_casual_chart(data, title, year_filter):

    # set Filter to 2010
    filtered_Data = data[data['timePeriod'] == year_filter]

    # Scatter Plot using plotly.express
    fig = px.bar(filtered_Data, x='country', y='absolute_value', 
                    title=title,
                    labels={'Time_Period': 'Time Period (Years)', 'absolute_value': 'Amount of vessels'})
    # logahritmic scale:
    fig.update_yaxes(type='log')
    fig.update_layout(
        xaxis=dict(tickangle=45)  # Rotates x-axis labels by 45 degrees
    )
    # Show the plot
    fig.show()

# Determining continent:
def load_countries(file):
    with open(file, 'r') as file:
        countries = file.read().splitlines()
    return countries

def sort_Continent(countries):
    # Sorting Countries to Continents
    asia_list = load_countries('asia.txt')
    europe_list = load_countries('europe.txt')
    america_list = load_countries('america.txt')
    australia_list = load_countries('australia.txt')
    southAmerica_list = load_countries('southAmerica.txt')
    continent = []
    for country in countries:
        if country in europe_list:
            continent.append('Europe')
        elif country in asia_list:
            continent.append('Asia')
        elif country in america_list:
            continent.append('America')
        elif country in australia_list:
            continent.append('Australia')
        elif country in southAmerica_list:
            continent.append('South America')
        else:
            continent.append('Unknown')
    return continent


# Initializing of CSV & DataFrames
FishBoats = pd.read_csv('Fish_Boote.csv')
FishInland = pd.read_csv('Fish_Inland.csv')

# FISH BOATS
# transfering mentionable values into lists
boat_referenceArea = FishBoats['Reference area'].tolist()
boat_timePeriod = FishBoats['TIME_PERIOD'].tolist()
boat_obsValue = FishBoats['OBS_VALUE'].tolist()

# Creating DataFrame
boat_Data = pd.DataFrame({
    'country': boat_referenceArea,
    'absolute_value': boat_obsValue,
    'timePeriod': boat_timePeriod
})
# FISH INLAND

# Transfering mentionable values into lists
inland_referenceArea = FishInland['Reference area'].tolist()
inland_timePeriod = FishInland['TIME_PERIOD'].tolist()
inland_obsValue = FishInland['OBS_VALUE'].tolist()

# Creating DataFrame
inland_Data = pd.DataFrame({
    'country': inland_referenceArea,
    'absolute_value': inland_obsValue,
    'timePeriod': inland_timePeriod
})


# Create chart for Boats
#create_casual_chart(boat_Data,"Scatter Plot of amount of fishing vessels in each Country for 2010",2010)
# Create chart for catches
#create_casual_chart(inland_Data,"Scatter Plot of amount of fishing vessels in each Country for 2010",2010)

# Mixed Scatter Plot

# finding common countries & time:
common_countries = set(boat_Data['country']).intersection(inland_Data['country'])
common_time = set(boat_Data[boat_Data["country"].isin(common_countries)]['timePeriod']
                    ).intersection(inland_Data[inland_Data["country"].isin(common_countries)]['timePeriod'])

# Creating a full index of common countries and time periods
full_index = pd.MultiIndex.from_product(
    [common_countries, common_time], names=["country", "timePeriod"]
)

# Reindexing both datasets and filling missing values with 0
boat_Data_final = (
    boat_Data.set_index(["country", "timePeriod"])
    .reindex(full_index)
    .fillna(0)
    .reset_index()
)
inland_Data_final = (
    inland_Data.set_index(["country", "timePeriod"])
    .reindex(full_index)
    .fillna(0)
    .reset_index()
)

# Merging the two DataFrames
combined_data = pd.merge(
    boat_Data_final, 
    inland_Data_final, 
    on=["country", "timePeriod"], 
    suffixes=("_boat", "_inland")  # Add suffixes to distinguish columns
)


combined_data['continent'] = sort_Continent(combined_data['country'].tolist())


# Filter to 2010
combined_data_timefiltered = combined_data[combined_data['timePeriod'] == 2010]

 # Scatter Plot using plotly.express
fig = px.scatter(
    combined_data_timefiltered, 
    x='absolute_value_boat', 
    y='absolute_value_inland', 
    title="Amount of Vessels vs Amount of Inland Fish Catches in 2010",
    labels={'absolute_value_boat': 'Amount of vessels', 'absolute_value_inland': 'Amount of catches'},
    color='continent',
    hover_data={
        'continent': False,
        'country': True,  # Show the country name
        'timePeriod': False,  # Hide the time period
        'absolute_value_boat': True,  # Show boats value
        'absolute_value_inland': True  # Show inland value
    }
    )

# logahritmic scale:
fig.update_yaxes(type='log')
fig.update_xaxes(type='log')
fig.update_layout(
    xaxis=dict(tickangle=45)  # Rotates x-axis labels by 45 degrees
 )
# Show the plot
fig.show()



# Correlation Coefficient Matrix
correlation_data = pd.DataFrame({
    'absolute_value_boat': combined_data_timefiltered['absolute_value_boat'],
    'absolute_value_inland': combined_data_timefiltered['absolute_value_inland']
})
print(f"Correlation Coefficient: \n {correlation_data.corr()}")


Correlation Coefficient: 
                        absolute_value_boat  absolute_value_inland
absolute_value_boat               1.000000               0.864841
absolute_value_inland             0.864841               1.000000
